# MVD 5. cvičení

## 1. část - TF-IDF s word embeddingy

V minulém cvičení bylo za úkol implementovat TF-IDF algoritmus nad datasetem z Kagglu. Dnešní cvičení je rozšířením této úlohy s použitím word embeddingů. Lze použít předtrénované GloVe embeddingy ze 3. cvičení, nebo si v případě zájmu můžete vyzkoušet práci s Word2Vec od Googlu (najdete [zde](https://code.google.com/archive/p/word2vec/)).

Cvičení by mělo obsahovat následující části:
- Načtení článků a embeddingů
- Výpočet document vektorů pomocí TF-IDF a word embeddingů 
    - Pro výpočet TF-IDF využijte [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) z knihovny sklearn
    - Vážený průměr GloVe / Word2Vec vektorů

<center>
$
doc\_vector = \frac{1}{|d|} \sum\limits_{w \in d} TF\_IDF(w) glove(w)
$
</center>

- Dotaz bude transformován stejně jako dokument

- Výpočet relevance pomocí kosinové podobnosti
<center>
$
score(q,d) = cos\_sim(query\_vector, doc\_vector)
$
</center>

### Načtení článků

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('../cv04/articles.csv', delimiter=',')
df.head()

,author,claps,reading_time,link,title,text
0,Justin Lee,8.3K,11,https://medium.com/swlh/chatbots-were-the-next...,Chatbots were the next big thing: what happene...,"Oh, how the headlines blared:\nChatbots were T..."
1,Conor Dewey,1.4K,7,https://towardsdatascience.com/python-for-data...,Python for Data Science: 8 Concepts You May Ha...,If you’ve ever found yourself looking up the s...
2,William Koehrsen,2.8K,11,https://towardsdatascience.com/automated-featu...,Automated Feature Engineering in Python – Towa...,Machine learning is increasingly moving from h...
3,Gant Laborde,1.3K,7,https://medium.freecodecamp.org/machine-learni...,Machine Learning: how to go from Zero to Hero ...,If your understanding of A.I. and Machine Lear...
4,Emmanuel Ameisen,935,11,https://blog.insightdatascience.com/reinforcem...,Reinforcement Learning from scratch – Insight ...,Want to learn about applied Artificial Intelli...


### Načtení embeddingů

In [6]:
import numpy as np
from tqdm import tqdm

with open('glove/glove.6B.50d.txt', 'r') as file:
    lines = file.readlines()
    words, vectors = [], np.zeros((len(lines), len(lines[0].strip().split(' ')) - 1))
    i = 0
    for line in tqdm(lines, 'Loading vectors'):
        word, *vector = line.strip().split(' ')
        words.append(word)
        vectors[i, :] = np.array(vector).astype(np.float64)
        i += 1
np.array(words).shape, vectors.shape

Loading vectors: 100%|██████████| 400001/400001 [00:22<00:00, 17512.86it/s]


((400001,), (400001, 50))

In [7]:
word2idx = {k: v for v, k in enumerate(words)}

### TF-IDF + Word2Vec a vytvoření doc vektorů

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [52]:
def make_doc_vector(document):
    vectorizer = TfidfVectorizer()
    scores = vectorizer.fit_transform([document])
    tf_idf = pd.DataFrame(scores.toarray(), columns=vectorizer.get_feature_names())

    vec = np.zeros((1, vectors.shape[1]))
    for word in vectorizer.get_feature_names():
        try:
            vec += vectors[word2idx[word]] * tf_idf[word][0]
        except:
            pass

    return vec / len(vectorizer.get_feature_names())

In [53]:
make_doc_vector(df.title[1])

/Users/admin/opt/anaconda3/envs/MVD_2021/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


array([[ 0.11297326,  0.01671256,  0.0067473 , -0.02334926,  0.09781586,
         0.02775704, -0.16440604, -0.23764566, -0.01647444,  0.06283889,
         0.06001796,  0.15566989, -0.05849096,  0.00570791,  0.15809441,
         0.0830353 ,  0.00769511,  0.04670637, -0.05362259, -0.09813815,
         0.02883481,  0.0620337 ,  0.13047074,  0.01771267,  0.12535656,
        -0.36996681, -0.22786137, -0.07816489,  0.05963644, -0.13055852,
         0.97595556, -0.01628415, -0.09973481, -0.22697778,  0.01069873,
         0.06329685, -0.01700181,  0.02438007,  0.0061157 , -0.04410222,
        -0.00136489, -0.05308185,  0.00820659,  0.17844667, -0.03375481,
         0.04537089,  0.11061478,  0.11117926, -0.0517525 ,  0.00865874]])

In [58]:
doc_vectors = np.zeros((len(df), vectors.shape[1]))
for i in tqdm(range(len(df)), 'Creating vectors'):
    doc_vectors[i] = make_doc_vector(df.iloc[i].title)

Creating vectors:   0%|          | 0/337 [00:00<?, ?it/s]/Users/admin/opt/anaconda3/envs/MVD_2021/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
Creating vectors: 100%|██████████| 337/337 [00:01<00:00, 229.61it/s]


### Transformace dotazu a výpočet relevance

In [59]:
def cossim(a, b):
    return np.dot(a, b.T) / (np.linalg.norm(a) * np.linalg.norm(b))

In [64]:
scores = np.zeros((len(df), 1))

query_vector = make_doc_vector('coursera vs udacity machine learning')
for i in tqdm(range(len(df)), 'Evaluating vectors'):
    scores[i] = cossim(query_vector, make_doc_vector(df.iloc[i].title))
df['score'] = scores

/Users/admin/opt/anaconda3/envs/MVD_2021/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
Evaluating vectors:   0%|          | 0/337 [00:00<?, ?it/s]/Users/admin/opt/anaconda3/envs/MVD_2021/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  
Evaluating vectors: 100%|██████████| 337/337 [00:01<00:00, 234.56it/s]


In [65]:
df.sort_values(by='score', ascending=False).head(10)[['title', 'score']]

,title,score
276,Coursera vs Udacity for Machine Learning – Hac...,0.893807
53,"Machine Learning เรียนอะไร, รู้ไปทําไม – O v e...",0.801541
144,A Beginner’s Guide to AI/ML 🤖👶 – Machine Learn...,0.771436
196,A Beginner’s Guide to AI/ML 🤖👶 – Machine Learn...,0.771436
68,A Beginner’s Guide to AI/ML 🤖👶 – Machine Learn...,0.771436
288,ИИ-психопат и ИИ-обманщик – Hey Machine Learning,0.753258
6,An intro to Machine Learning for designers – U...,0.749931
90,An intro to Machine Learning for designers – U...,0.749931
192,Ultimate Guide to Leveraging NLP & Machine Lea...,0.738272
49,The 7 Best Data Science and Machine Learning P...,0.735582


## Bonus - Našeptávání

Bonusem dnešního cvičení je našeptávání pomocí rekurentních neuronových sítí. Úkolem je vytvořit jednoduchou rekurentní neuronovou síť, která bude generovat text (character-level přístup). 

Optimální je začít po dokončení cvičení k předmětu ANS, kde se tato úloha řeší. 

Dataset pro učení vaší neuronové sítě naleznete na stránkách [Yahoo research](https://webscope.sandbox.yahoo.com/catalog.php?datatype=l&guccounter=1), lze využít např. i větší [Kaggle dataset](https://www.kaggle.com/c/yandex-personalized-web-search-challenge/data) nebo vyhledat další dataset na [Google DatasetSearch](https://datasetsearch.research.google.com/).

Vstupem bude rozepsaný dotaz a výstupem by měly být alespoň 3 dokončené dotazy.